In [1]:
from graphdatascience import GraphDataScience

gds = GraphDataScience("bolt://localhost:7687", auth=None)

/Users/pax-data/.local/share/virtualenvs/neo4j_companies_knn-8Hv7Sjok/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gds.set_database("neo4j")

In [3]:
G, metadata = gds.graph.project( "donations", ["Company", "Recipient"],
                        { 'DONATED': {'properties': 'value' }})

In [4]:
metadata

nodeProjection            {'Company': {'label': 'Company', 'properties':...
relationshipProjection    {'DONATED': {'orientation': 'NATURAL', 'aggreg...
graphName                                                         donations
nodeCount                                                             26235
relationshipCount                                                        65
projectMillis                                                           254
Name: 0, dtype: object

In [5]:
gds.graph.list()

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,"{'p99': 0, 'min': 0, 'max': 15, 'mean': 0.0024...",donations,neo4j,30 MiB,31679288,26235,65,{'relationshipProjection': {'DONATED': {'orien...,9.444257e-08,2022-09-14T14:26:08.745964000+00:00,2022-09-14T14:26:09.001856000+00:00,"{'graphProperties': {}, 'relationships': {'DON..."


In [58]:
gds.graph.get("donations").drop()

graphName                                                    donations
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                        26235
relationshipCount                                                   65
configuration        {'relationshipProjection': {'DONATED': {'orien...
density                                                            0.0
creationTime                       2022-09-14T10:50:23.530036000+00:00
modificationTime                   2022-09-14T10:50:40.530654000+00:00
schema               {'graphProperties': {}, 'relationships': {'DON...
Name: 0, dtype: object

In [6]:
result = gds.fastRP.mutate(
    G,
    mutateProperty="embedding",
    embeddingDimension=4,
    randomSeed=56,
    relationshipWeightProperty="value",
    iterationWeights=[0.8, 1, 1, 1],
)

In [7]:
result_knn = gds.knn.write(
    G,
    topK=2,
    nodeProperties=["embedding"],
    writeRelationshipType="SIMILAR",
    writeProperty="score",
)
print(f"Relationships produced: {result_knn['relationshipsWritten']}")
print(f"Nodes compared: {result_knn['nodesCompared']}")

Relationships produced: 52470
Nodes compared: 26235


In [13]:
query = """
        MATCH (c1:Company {name: "BLOOMBERG UK LIMITED"})-[r:SIMILAR]->(c2:Company)
        WHERE c1.name <> 'None' AND c2.name <> 'None'
        RETURN c1.name, c1.companyNumber, c2.name, c2.companyNumber
        LIMIT 1
        """

gds.run_cypher(query)

,c1.name,c1.companyNumber,c2.name,c2.companyNumber
0,BLOOMBERG UK LIMITED,03430322,TRAFALGAR PUB LIMITED,07983771


In [14]:
query="""
MATCH (p:Person)-[:HAS_CONTROL]->(c:Company {name: "MAGNETAR FINANCIAL (UK) LLP"})
RETURN p.name
"""
gds.run_cypher(query)

,p.name
0,Alec Litowitz
1,Ross Steven Laser
